CODE BLOCK TO EXTRACT WEATHER FROM LAT LONG

In [1]:
import pandas as pd
import geocoder
import reverse_geocode as rg
import numpy as np

In [2]:
LIMITED = True
METEOSTAT = False

In [3]:
#icouk_data_file = '..\\Data\\icouk_sighting_data_with_params.csv'
icop_data_file = '..\\Data\\icop_ahmed_2020_sighting_data_with_params.csv'
#alrefay_data_file = '..\\Data\\alrefay_2018_sighting_data_with_params.csv'
#allawi_data_file = '..\\Data\\schaefer_odeh_allawi_2022_sighting_data_with_params.csv'
#yallop_data_file = '..\\Data\\yallop_sighting_data_with_params.csv'

#icouk_data = pd.read_csv(icouk_data_file)
#icop_data = pd.read_csv(icop_data_file)
#alrefay_data = pd.read_csv(alrefay_data_file)
#allawi_data = pd.read_csv(allawi_data_file)
#yallop_data = pd.read_csv(yallop_data_file)


#data = pd.concat([icouk_data,icop_data,alrefay_data,yallop_data])
data = pd.read_csv(icop_data_file)


#data = data[['Latitude', "Longitude", 'Date']]
if LIMITED:
    df = data.copy('Deep').head(20)
else:
    df = data.copy('Deep')

In [4]:
city_arr = []
city_dic = rg.search(df[['Latitude', 'Longitude']])
for i in range(len(city_dic)):
    city_arr.append((city_dic[i])['city']+','+(city_dic[i])['country_code'])

df['City'] = city_arr

In [5]:
df

,Index,Date,Latitude,Longitude,Moon Age,Sunset,Moonset,Lag,Moon Alt,Moon Az,...,W',q,q',Cloud Level,Seen,Method,Methods,Visibility,Source,City
0,0,2.455182e+06,10.62283,10.02878,0.208,2.455182e+06,2.455182e+06,2.75000,-0.24016,243.84099,...,0.01721,-1.08601,-1.08601,0.00,Not_seen,Not_seen,Not_seen,0.0,ICOP,"Zalanga,NG"
1,1,2.455182e+06,26.43678,50.10400,0.072,2.455182e+06,2.455182e+06,-8.55000,-1.46466,241.42564,...,0.01707,-1.03339,-1.03340,0.00,Not_seen,Not_seen,Not_seen,0.0,ICOP,"Dammam,SA"
2,2,2.455182e+06,-33.92899,18.41740,0.249,2.455182e+06,2.455182e+06,13.06667,0.68812,239.97115,...,0.01503,-0.91727,-0.91727,0.00,Not_seen,Not_seen,Not_seen,0.0,ICOP,"Cape Town,ZA"
3,3,2.455182e+06,10.55202,-61.37228,0.410,2.455182e+06,2.455182e+06,13.15000,0.99048,243.76188,...,0.04129,-0.84373,-0.84372,0.00,Not_seen,Not_seen,Not_seen,0.0,ICOP,"Chaguanas,TT"
4,4,2.455182e+06,37.22966,-80.41368,0.421,2.455182e+06,2.455182e+06,5.96667,-0.03313,236.67397,...,0.05152,-1.02520,-1.02520,0.75,Not_seen,Not_seen,Not_seen,0.0,ICOP,"Plum Creek,US"
5,5,2.455183e+06,32.48586,3.67710,1.210,2.455183e+06,2.455183e+06,51.15000,4.36770,236.88158,...,0.36924,-0.00918,-0.00892,0.75,Seen,Seen_eye,Seen_eye;Seen_binoculars;Seen_telescope;Seen_ccd,1.0,ICOP,"Ghardaïa,DZ"
6,6,2.455183e+06,23.81065,90.41265,0.979,2.455183e+06,2.455183e+06,40.28333,3.77499,240.49020,...,0.23647,-0.20482,-0.20467,0.00,Not_seen,Not_seen,Not_seen,0.0,ICOP,"Paltan,BD"
7,7,2.455183e+06,10.38253,7.85332,1.231,2.455183e+06,2.455183e+06,55.08333,6.00264,243.83381,...,0.37079,0.27805,0.27841,0.00,Seen,Seen_eye,Seen_eye;Seen_binoculars;Seen_telescope;Seen_ccd,1.0,ICOP,"Kajuru,NG"
8,8,2.455183e+06,6.45506,3.39418,1.249,2.455183e+06,2.455183e+06,56.28333,6.24792,244.63284,...,0.38076,0.32595,0.32633,0.75,Not_seen,Not_seen,Not_seen,0.0,ICOP,"Lagos,NG"
9,9,2.455183e+06,6.61912,3.50413,1.248,2.455183e+06,2.455183e+06,56.25000,6.24096,244.60141,...,0.38047,0.32442,0.32479,0.75,Not_seen,Not_seen,Not_seen,0.0,ICOP,"Ikorodu,NG"


In [6]:
if METEOSTAT:
    # Import Meteostat library and dependencies
    from datetime import datetime,timedelta
    from astropy.time import Time
    from meteostat import Point, Hourly

    weather_data =[]
    for i in range(len(city_dic)):
        day = Time(df['Date'][i],format='jd').to_datetime()
        weather_data_temp = Hourly(Point(df['Latitude'][i],df['Longitude'][i]), day-timedelta(hours=0.5), day + timedelta(hours=0.5)) #day+timedelta(days=1))
        x = weather_data_temp.fetch()
        if (len(x)>0):
            weather_data.append(x)
        else:
            weather_data.append( pd.DataFrame(np.zeros([1, 11])*np.nan, columns = ['temp', 'dwpt', 'rhum', 'prcp', 'snow','wdir ','wspd','wpgt','pres', 'tsun', 'coco']))
    weather_dataset = (pd.concat(weather_data, ignore_index=True)).drop(['wdir','wspd', 'wpgt'], axis=1)

In [7]:
import requests
from bs4 import BeautifulSoup
import re


In [13]:
url = 'https://www.weatheronline.co.uk/weather/maps/current?LANG=en&UP=0&R=310&TYP=bedeckung&ART=tabelle&LANG=en&DATE=1699315200&KEY=ukuk&LAND=ukuk&CONT=ukuk&SORT=2&SI=mph&CEL=C&UD=0'
response = requests.get(url)
print(response.status_code)

200


In [14]:
soup = BeautifulSoup(response.text, 'html.parser')

In [15]:
table = soup.find('table', class_ = 'gr3')

table = str(table)
#print(table)

In [11]:
#for datapoint in table.find_all('tr'):
#    #print(datapoint)
#    rows = datapoint.find_all('td')
#print(rows)

In [19]:
names = re.findall("(?<=CEL=C\">)(.*?)(?= \()", table)
cloud_level = re.findall("(?<=<td>)(\d)(?=<\/td>)", table)
print(names)
print(len(cloud_level))

['Manston', 'Tibenham Airport', 'Wattisham Airport', 'Weybourne', 'Shoeburyness', 'Marham', 'Andrewsfield Aerodrome', 'Wainfleet All Saints', 'Herstmonceux', 'Donna Nook', 'Holbeach Air Weapons Range', 'Kenley', 'Heathrow Airport', 'Bridlington', 'Coningsby', 'Charlwood', 'Wittering', 'Northolt', 'Leconfield', 'Bedford', 'Cranwell', 'Waddington', 'Baltasound', 'Thorney Island', 'RAF Odiham', 'Benson', 'Lerwick', 'Loftus Samos', 'Nottingham/Watnall', 'Church Lawford', 'Topcliffe', 'Leeming', 'Middle Wallop', 'RAF Brize Norton', 'Boulmer', 'Fair Isle', 'Coleshill', 'RAF Little Rissington', 'MoD Boscombe Down', 'Larkhill', 'Bournemouth', 'Albemarle', 'Bingley', 'Thorncliffe', 'RAF Lyneham', 'Pershore', 'Aberdeen Airport', 'Jersey Airport', 'Redesdale Camp', 'Warcop Range', 'Rostherne', 'Isle of Portland', 'Inverbervie', 'Spadeadam II', 'Almondsbury', 'Guernsey Airport', 'Yeovil', 'Shawbury', 'Shap', 'Charterhall', 'Credenhill', 'Aboyne', 'Shobdon', 'Leuchars', 'Kirkwall Airport', 'Hawarde